In [1]:
from bs4 import BeautifulSoup
import requests
import re
import time
import pandas as pd
from selenium import webdriver
import pandas as pd

In [2]:
from selenium import webdriver
import pandas as pd

In [3]:
def load_webpage():
    driver = webdriver.Chrome('/usr/bin/chromedriver')
    driver.get('https://nces.ed.gov/collegenavigator/')
    return driver

In [4]:
def get_search_options():
    el = driver.find_element_by_id('MapWrap')
    options = el.find_elements_by_tag_name('option')
    return options

In [5]:
def search_for_option(no_pref, option):
    no_pref.click() #deselect no preference
    option.click()
    bachelors_button = driver.find_element_by_id('ctl00_cphCollegeNavBody_ucSearchMain_chkBach')
    bachelors_button.click()
    search_button = driver.find_element_by_id('ctl00_cphCollegeNavBody_ucSearchMain_btnSearch')
    search_button.click()
    return

In [6]:
def get_student_pop():
    try:
        student_pop = driver.find_element_by_css_selector("#RightContent > div.dashboard > div > div:nth-child(2) > table > tbody > tr:nth-child(7) > td.srb + td")
        student_pop = re.findall('\d+', student_pop.text.replace(',',''))[0]
    except:
        student_pop = None
    return student_pop

In [7]:
def get_college_data_row():
    #right now I just grab the name and student population.
    #ill probably have individual funcs for grabbing each param later.
    college_data_row = []
    name = driver.find_element_by_css_selector(".headerlg").text
    student_pop = get_student_pop()
    student_fac_ratio = find_student_faculty_ratio()
    setting = find_campus_setting()
    IPEDS_ID = get_IPEDS_ID()
    expand_all()
    perc_aid = get_percent_aid()
    religious_aff = get_religious_aff()
    perc_reg_disabled = get_registered_disability_perc()
    full_time_fac, part_time_fac = get_faculty_full_part()
    criminal_offenses, vawa_offenses, arrests, disciplinary_actions = get_college_offense_data()
    avg_default_rate = get_avg_default_rate()
    college_data_row.extend([name, student_pop, student_fac_ratio,
                             setting, IPEDS_ID, perc_aid, religious_aff,
                             perc_reg_disabled, full_time_fac, part_time_fac,
                            criminal_offenses, vawa_offenses, arrests, disciplinary_actions,
                            avg_default_rate])
    print(college_data_row)
    return college_data_row

In [8]:
def get_data_on_page():
    data = []
    rows = driver.find_elements_by_css_selector(".resultsTable > tbody > tr")
    for ix, row in enumerate(rows):
        college_link = (driver
               .find_elements_by_css_selector(".resultsTable > tbody > tr")[ix]
               .find_element_by_css_selector("a"))
        college_link.click()
        row = get_college_data_row()
        data.append(row)
        driver.back()
    return data

In [9]:
def next_page(selected_page_num):
    css_addition = ' + a'*selected_page_num
    try:
        driver.find_element_by_css_selector('.colorful' + css_addition).click()
        return True
    except:
        return False

In [10]:
def get_data_for_all_pages():
    data = get_data_on_page()
    page_num = 1
    can_go_to_next_page = next_page(page_num)
    while can_go_to_next_page == True:
        data.extend(get_data_on_page())
        page_num += 1
        can_go_to_next_page = next_page(page_num)
    return data

In [11]:
def get_all_college_data(driver):
    option_ix = 1
    college_data = []
    more_options = True
    while more_options:
        options = get_search_options()
        no_pref = options[0]
        try:
            option = options[option_ix]
            search_for_option(no_pref, option)
            college_data.extend(get_data_for_all_pages())
            driver.get('https://nces.ed.gov/collegenavigator/')
            option_ix += 1
        except:
            more_options = False
    return college_data

In [12]:
def get_info_table():
    info_table = driver.find_element_by_css_selector('#RightContent > div.dashboard > div')
    return info_table

In [13]:
def find_number(text, c):
    return re.search(r'%s(\d+)' % c, text).group(1)

In [14]:
def find_student_faculty_ratio():
    try:
        info = get_info_table().text
        string = re.search('Student-to-faculty ratio:(.*)', info).group(1)
        ratio = eval(string.strip().replace(' to ', '/'))
        return ratio
    except:
        return None

In [15]:
def find_campus_setting():
    try:
        info = get_info_table().text
        setting = re.search('Campus setting:   (.*)\n', info).group(1)
        return setting
    except:
        return None

In [16]:
def get_IPEDS_ID():
    try:
        info = get_info_table().text
        url = str(driver.current_url)
        ID = int(find_number(text = url, c = 'id='))
        return ID
    except:
        return None

In [17]:
def expand_all():
    pointer = driver.find_element_by_css_selector('#RightContent > div.fadeyell > div > a:nth-child(1)')
    pointer.click()
    return

In [18]:
def get_percent_aid():
    table = get_all_table()
    evals =["re.search('Any student (.*)%', table).group(1).split(' ')[-1]",
            "re.search('\d+,\d+ (.*) —— ——\nGrant or scholarship aid', table).group(1)"]
    for evaluation in evals:  
        try:
            aid_perc = eval(evaluation)
            return aid_perc
        except:
            continue
    else:
        return None
    

In [19]:
def get_religious_aff():
    table = get_all_table()
    try:
        aff = re.search('Religious Affiliation\n(.*)\n', table).group(1)
        return aff
    except:
        return None

In [20]:
def get_registered_disability_perc():
    table = get_all_table()
    try:
        disability_perc = re.search('with office of disability services\n(.*)\n', table).group(1)
        return disability_perc
    except:
        return None

In [21]:
def get_all_table():
    table = driver.find_element_by_css_selector('#RightContent').text
    return table

In [22]:
def get_faculty_full_part():
    table = get_all_table()
    try:
        fac = re.search('\nTotal faculty(.*)\n', table).group(1)
        fac = fac.split(' ')
        full_time = fac[1]
        part_time = fac[2]
        return full_time, part_time
    except:
        return None, None

In [23]:
def get_college_offense_data():
    '''
    Returns criminal_offenses, vawa_offeneses, arrests, disciplinary_actions
    
    
    sum of Criminal_Offenses, VAWA Offenses, Arrests, Disciplinary Actions from 2015,2016,2017.
    Offenses include Murder, Negligent Manslaughter, Rape, Fondling, Incest??,
    Statutory Rape, Robbery, Assault, Burglary, Vehicle Theft, Arson. 
    Vawa Offences are domestic violoence, dating violence, stalking.
    Arrests are for weapons, possesion, drug abuse, liquor law violations.
    Disciplinary are for weapons, possesion, drug abuse, liquor law violations.
    '''
    try:
        table = driver.find_element_by_css_selector('#divctl00_cphCollegeNavBody_ucInstitutionMain_ctl11 > div > table > tbody').text
        table = table.replace('2017', '')
        table = table.replace('2016', '')
        table = table.replace('2015', '')
        nums = [int(num) for num in table if num.isnumeric()]
        criminal_offenses = sum(nums[0:33])
        vawa_offenses = sum(nums[33:42])
        arrests = sum(nums[42:51])
        disciplinary_actions = sum(nums[51:])
        return criminal_offenses, vawa_offenses, arrests, disciplinary_actions
    except:
        return None, None, None, None
    

In [24]:
def get_avg_default_rate():
    '''Returns avg default rate from 2016, 2015, 2014'''
    table = driver.find_element_by_css_selector('#fedloans').text
    try:
        rates = re.search('Default rate (.*)\nNumber in default', table).group(1).replace('%', '').split(' ')
        rate_nums = [float(rate) for rate in rates]
        avg_rates = sum(rate_nums)/3
        return round(avg_rates,2)
    except:
        return None

In [25]:
driver = load_webpage()

In [27]:
get_all_college_data(driver)

['Alabama A & M University', '6106', 20.0, 'City: Midsize', 100654, '90', 'Not applicable', '3% or less', '259', '95', 22, 14, 24, 62, 18.97]


[]